In [1]:
import os
import pandas as pd

from datetime import datetime

## Load COMETKiwi 2022 model

In [ ]:
# log into Huggingface hub
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/rjersakova/.cache/huggingface/token
Login successful


In [3]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-cometkiwi-da")
model = load_from_checkpoint(model_path)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
Encoder model frozen.


## Make predictions

In [4]:
# path to ARC-MTQE directory
main_dir = os.path.dirname(os.getcwd())

In [5]:
data_dir = os.path.join(main_dir, "mlqe-pe", "data", "catastrophic_errors")

In [6]:
# save results here
now = datetime.today().strftime("%Y%m%dT%H%M%S")
out_dir = os.path.join(main_dir, "results", "comet_kiwi", now)
os.makedirs(out_dir, exist_ok=True)

In [7]:
# make predictions for all language pairs listed here
language_pairs = ["encs", "ende", "enja", "enzh"]

In [8]:
for lp in language_pairs:

    # load data
    path_data = os.path.join(data_dir, f"{lp}_majority_test_blind.tsv")
    df_data = pd.read_csv(path_data, sep='\t', header=None, names=["idx", "source", "target"])
    
    # format for COMETKiwi input: [{"src":"...", "mt":"..."}, {...}] 
    comet_data = []
    for i, row in df_data.iterrows():
        comet_data.append({"src": row["source"], "mt": row["target"]})

    # predict
    model_output = model.predict(comet_data, batch_size=8, gpus=0)

    # save output
    df_results = pd.DataFrame({"idx": df_data["idx"], "comet_score": model_output.scores})
    df_results.to_csv(os.path.join(out_dir, f"{lp}_predictions.csv"), index=False)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 125/125 [03:21<00:00,  1.61s/it]
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 125/125 [03:22<00:00,  1.62s/it]
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 125/125 [05:30<00:00,  2.65s/it]
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 125/125 [03:40<00:00,  1.77s/it]
